# Notebook to store customers records (JSON) into bronze.customers table

Define constant variables (location, target entity)

In [0]:
FILES_LOCATION= '/Volumes/workspace/default/staging/customers/'
TARGET_SCHEMA="bronze"
TARGET_TABLE="customers"

### Getting the customers (parquet)

Get all *.parquet file in `FILES_LOCATION`

In [0]:
files=dbutils.fs.ls(FILES_LOCATION)

customer_files=[f.path for f in files if f.path.endswith(".parquet")]

### Read *.parquet files and load them into a DataFrame

In [0]:
df_customers=spark.read.parquet(*customer_files)
df_customers.display()


Add ingestion timestamp

In [0]:
from pyspark.sql.functions import current_timestamp

df_customers=df_customers.withColumn("ingestime", current_timestamp())

columns = ["ingestime"] + [col for col in df_customers.columns if col != "ingestime"]
df_customers = df_customers.select(columns)

df_customers.display()

### Create Schema in Catalog

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {TARGET_SCHEMA}")

### Write into bronze.customers

In [0]:
df_customers.write.partitionBy("ingestime").format("delta").mode("overwrite").saveAsTable(f"{TARGET_SCHEMA}.{TARGET_TABLE}")